In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Configuración visual mejorada
plt.style.use('seaborn-v0_8')
sns.set_palette("Set2")
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'

print("📊 ANÁLISIS DE OCUPACIÓN EDUCATIVA - SOLICITUDES VS CAPACIDAD")
print("="*65)


📊 ANÁLISIS DE OCUPACIÓN EDUCATIVA - SOLICITUDES VS CAPACIDAD


In [2]:
# === CARGA DE DATOS ===
def load_education_data():
    """Carga solo los datasets educativos necesarios"""
    data = {}

    # Datos educativos principales
    files = {
        'education': '../downloads/normalizacion/education.csv',
        'admissions': '../downloads/normalizacion/education_admition.csv',
        'enrollment': '../downloads/normalizacion/education_enrollment.csv',
        'edu_municipality': '../downloads/normalizacion/education_municipality.csv',
        'municipality': '../downloads/normalizacion/municipality.csv'
    }

    for name, path in files.items():
        delimiter = ';' if 'education' in path and 'municipality' not in path else ','
        data[name] = pd.read_csv(path, delimiter=delimiter)
        # Convertir IDs a string para consistencia
        if 'id' in data[name].columns:
            data[name]['id'] = data[name]['id'].astype(str)
        if 'id_education' in data[name].columns:
            data[name]['id_education'] = data[name]['id_education'].astype(str)
        if 'id_municipality' in data[name].columns:
            data[name]['id_municipality'] = data[name]['id_municipality'].astype(str)

    return data

data = load_education_data()
print(f"✅ Datos educativos cargados:")
for key, df in data.items():
    print(f"   • {key}: {len(df):,} registros")


✅ Datos educativos cargados:
   • education: 1,978 registros
   • admissions: 39,297 registros
   • enrollment: 24,159 registros
   • edu_municipality: 2,780 registros
   • municipality: 179 registros


In [3]:
# === CONFIGURACIÓN DE CICLOS EDUCATIVOS ===
def get_cycle_config():
    """Configuración de ciclos con años de permanencia"""
    return {
        'infantil_i_ciclo': {
            'name': 'Infantil I',
            'años_permanencia': 3  # 0-2 años
        },
        'infantil_ii_ciclo': {
            'name': 'Infantil II',
            'años_permanencia': 3  # 3-5 años
        },
        'primaria': {
            'name': 'Primaria',
            'años_permanencia': 6  # 6-11 años
        },
        'eso': {
            'name': 'ESO',
            'años_permanencia': 4  # 12-15 años
        }
    }

cycle_config = get_cycle_config()
print("📚 Configuración de ciclos educativos:")
for cycle, config in cycle_config.items():
    print(f"   • {config['name']}: {config['años_permanencia']} años de permanencia")


📚 Configuración de ciclos educativos:
   • Infantil I: 3 años de permanencia
   • Infantil II: 3 años de permanencia
   • Primaria: 6 años de permanencia
   • ESO: 4 años de permanencia


In [4]:
# === ANÁLISIS DE OCUPACIÓN POR CENTRO EDUCATIVO ===
def analyze_center_occupancy(data, cycle_config):
    """Analiza ocupación de centros: solicitudes vs capacidad estimada"""

    target_year = "2022-2023"  # Año para comparación de ocupación actual

    # FILTRO INICIAL: Solo trabajar con los ciclos específicos
    valid_cycles = ['infantil_i_ciclo', 'infantil_ii_ciclo', 'primaria', 'eso']

    # Para admisiones, usar TODOS los años disponibles para encontrar el máximo
    admissions = data['admissions'].copy()
    # Filtrar solo los ciclos válidos
    admissions = admissions[admissions['cycle'].isin(valid_cycles)]

    # Para matrícula, usar TODOS los años disponibles para encontrar el máximo
    enrollment = data['enrollment'].copy()
    # Filtrar solo los ciclos válidos
    enrollment = enrollment[enrollment['cycle'].isin(valid_cycles)]

    print(f"🎯 Analizando ocupación actual para año: {target_year}")
    print(f"   • Ciclos incluidos: {valid_cycles}")
    print(f"   • Registros de admisiones (filtrados): {len(admissions):,}")
    print(f"   • Registros de matrícula (filtrados): {len(enrollment):,}")
    print(f"   • Años disponibles: {sorted(set(enrollment['year'].unique()) | set(admissions['year'].unique()))}")

    # FILTRO PREVIO: Solo analizar centros que tienen datos en los ciclos válidos
    centros_con_ciclos_validos = set()

    # Identificar centros que tienen datos de matrícula en ciclos válidos
    centros_enrollment = enrollment['id_education'].unique()
    centros_con_ciclos_validos.update(centros_enrollment)

    # Identificar centros que tienen datos de admisiones en ciclos válidos
    centros_admissions = admissions['id_education'].unique()
    centros_con_ciclos_validos.update(centros_admissions)

    print(f"   • Centros con datos en ciclos válidos: {len(centros_con_ciclos_validos)}")

    # Análisis por centro educativo
    center_analysis = []

    for center_id in centros_con_ciclos_validos:  # Solo analizar centros relevantes
        center_info = data['education'][data['education']['id'] == center_id]
        if center_info.empty:
            continue  # Saltar si no existe información del centro

        center_info = center_info.iloc[0]
        center_data = {
            'id_education': center_id,
            'center_name': center_info['name_short'],
            'center_type': center_info.get('description_short', 'No especificado')
        }

        # Variables para totales del centro
        total_plazas_estimadas = 0
        total_max_matriculados = 0
        total_matriculados_año_objetivo = 0
        total_solicitudes_presentadas = 0
        total_solicitudes_admitidas = 0
        ciclos_activos = 0

        # Analizar cada ciclo (solo los válidos)
        for cycle in valid_cycles:
            config = cycle_config[cycle]

            # y luego tomar el máximo

            # Obtener datos de matrícula para este ciclo en todos los años
            cycle_enrollment_all_years = enrollment[
                (enrollment['id_education'] == center_id) &
                (enrollment['cycle'] == cycle)
            ]

            # Obtener datos de admisiones para este ciclo en todos los años
            cycle_admissions_all_years = admissions[
                (admissions['id_education'] == center_id) &
                (admissions['cycle'] == cycle) &
                (admissions['type_solicitude'] == 'Admitidas')
            ]

            # Calcular plazas estimadas por año: (matriculados + admitidos) ÷ años_permanencia
            plazas_por_año = {}

            # Obtener todos los años que tienen datos
            años_matricula = set(cycle_enrollment_all_years['year'].unique())
            años_admisiones = set(cycle_admissions_all_years['year'].unique())
            todos_los_años = años_matricula | años_admisiones

            for año in todos_los_años:
                # Matriculados en este año
                matriculados_año = cycle_enrollment_all_years[
                    cycle_enrollment_all_years['year'] == año
                ]['total'].sum()

                # Admitidos en este año
                admitidos_año = cycle_admissions_all_years[
                    cycle_admissions_all_years['year'] == año
                ]['total'].sum()

                # Plazas estimadas para este año
                if matriculados_año > 0 or admitidos_año > 0:
                    plazas_año = matriculados_año / config['años_permanencia']
                    plazas_por_año[año] = {
                        'matriculados': matriculados_año,
                        'admitidos': admitidos_año,
                        'total_estudiantes': matriculados_año,
                        'plazas_estimadas': plazas_año
                    }

            # Tomar el MÁXIMO de plazas estimadas entre todos los años
            if plazas_por_año:
                año_max_plazas = max(plazas_por_año.keys(), key=lambda x: plazas_por_año[x]['plazas_estimadas'])
                max_plazas_data = plazas_por_año[año_max_plazas]

                plazas_estimadas = max_plazas_data['plazas_estimadas']
                max_matriculados = max_plazas_data['matriculados']
                max_admitidos = max_plazas_data['admitidos']
                max_total_estudiantes = max_plazas_data['total_estudiantes']
                año_max = año_max_plazas
            else:
                plazas_estimadas = 0
                max_matriculados = 0
                max_admitidos = 0
                max_total_estudiantes = 0
                año_max = 'N/A'

            # Datos del año objetivo para comparación de ocupación actual
            cycle_enrollment_target = enrollment[
                (enrollment['id_education'] == center_id) &
                (enrollment['cycle'] == cycle) &
                (enrollment['year'] == target_year)
            ]
            matriculados_año_objetivo = cycle_enrollment_target['total'].sum()

            # Solicitudes presentadas del año objetivo
            cycle_admissions_target = admissions[
                (admissions['id_education'] == center_id) &
                (admissions['cycle'] == cycle) &
                (admissions['year'] == target_year)
            ]

            solicitudes_presentadas = cycle_admissions_target[
                cycle_admissions_target['type_solicitude'] == 'Presentadas'
            ]['total'].sum()
            solicitudes_admitidas = cycle_admissions_target[
                cycle_admissions_target['type_solicitude'] == 'Admitidas'
            ]['total'].sum()

            if cycle == 'primaria':
                # Verificar si el centro también ofrece infantil II (usar máximo también)
                infantil_ii_enrollment_all = enrollment[
                    (enrollment['id_education'] == center_id) &
                    (enrollment['cycle'] == 'infantil_ii_ciclo')
                ]

                infantil_ii_admissions_all = admissions[
                    (admissions['id_education'] == center_id) &
                    (admissions['cycle'] == 'infantil_ii_ciclo') &
                    (admissions['type_solicitude'] == 'Admitidas')
                ]

                if not infantil_ii_enrollment_all.empty or not infantil_ii_admissions_all.empty:
                    # Calcular el máximo de infantil II usando la misma lógica
                    plazas_infantil_por_año = {}
                    años_inf_matricula = set(infantil_ii_enrollment_all['year'].unique())
                    años_inf_admisiones = set(infantil_ii_admissions_all['year'].unique())
                    todos_años_inf = años_inf_matricula | años_inf_admisiones

                    for año in todos_años_inf:
                        mat_inf = infantil_ii_enrollment_all[
                            infantil_ii_enrollment_all['year'] == año
                        ]['total'].sum()

                        if mat_inf > 0:
                            plazas_inf_año = (mat_inf) / cycle_config['infantil_ii_ciclo']['años_permanencia']
                            plazas_infantil_por_año[año] = plazas_inf_año

                    if plazas_infantil_por_año:
                        # Usar las plazas del año objetivo en lugar del máximo histórico
                        if target_year in plazas_infantil_por_año:
                            transiciones_automaticas = plazas_infantil_por_año[target_year]
                        else:
                            # Si no hay datos para el año objetivo, usar 0 o el último año disponible
                            transiciones_automaticas = 0

                        max_plazas_infantil = max(plazas_infantil_por_año.values())
                    else:
                        transiciones_automaticas = 0

                    # SOLICITUDES ADMITIDAS CORREGIDAS = Admisiones + Transiciones automáticas
                    solicitudes_admitidas_corregidas = solicitudes_admitidas + transiciones_automaticas
                    center_data[f'{cycle}_transiciones_infantil'] = transiciones_automaticas
                    center_data[f'{cycle}_max_plazas_infantil_ii'] = max_plazas_infantil if plazas_infantil_por_año else 0
                else:
                    solicitudes_admitidas_corregidas = solicitudes_admitidas
                    center_data[f'{cycle}_transiciones_infantil'] = 0
                    center_data[f'{cycle}_max_plazas_infantil_ii'] = 0
            else:
                solicitudes_admitidas_corregidas = solicitudes_admitidas
                center_data[f'{cycle}_transiciones_infantil'] = 0

            # Tasa de ocupación: matriculados actuales / máximo histórico matriculados
            tasa_ocupacion_ciclo = (matriculados_año_objetivo / max_matriculados) if max_matriculados > 0 else 0

            # Ratio demanda: solicitudes presentadas / plazas estimadas
            ratio_demanda_ciclo = (solicitudes_presentadas / plazas_estimadas) if plazas_estimadas > 0 else 0

            # Ratio admisión vs capacidad: solicitudes admitidas / plazas estimadas
            ratio_admision_vs_capacidad = (solicitudes_admitidas_corregidas / plazas_estimadas) if plazas_estimadas > 0 else 0

            # Para primaria, excluir las transiciones automáticas del cálculo de eficiencia
            eficiencia_admision = (solicitudes_admitidas / solicitudes_presentadas) if solicitudes_presentadas > 0 else 0

            # Guardar métricas del ciclo
            center_data.update({
                f'{cycle}_max_matriculados': max_matriculados,
                f'{cycle}_max_admitidos': max_admitidos,
                f'{cycle}_max_total_estudiantes': max_total_estudiantes,
                f'{cycle}_año_max': año_max,
                f'{cycle}_matriculados_objetivo': matriculados_año_objetivo,
                f'{cycle}_plazas_estimadas': plazas_estimadas,
                f'{cycle}_solicitudes_presentadas': solicitudes_presentadas,
                f'{cycle}_solicitudes_admitidas': solicitudes_admitidas,
                f'{cycle}_solicitudes_admitidas_corregidas': solicitudes_admitidas_corregidas,
                f'{cycle}_tasa_ocupacion': tasa_ocupacion_ciclo,
                f'{cycle}_ratio_demanda': ratio_demanda_ciclo,
                f'{cycle}_ratio_admision_capacidad': ratio_admision_vs_capacidad,
                f'{cycle}_eficiencia_admision': eficiencia_admision,
                f'{cycle}_activo': 1 if plazas_estimadas > 0 or solicitudes_presentadas > 0 else 0
            })

            # Acumular totales si el ciclo está activo
            if plazas_estimadas > 0 or solicitudes_presentadas > 0:
                ciclos_activos += 1
                total_plazas_estimadas += plazas_estimadas
                total_max_matriculados += max_matriculados
                total_matriculados_año_objetivo += matriculados_año_objetivo
                total_solicitudes_presentadas += solicitudes_presentadas
                total_solicitudes_admitidas += solicitudes_admitidas_corregidas

        # Solo incluir centros que tienen al menos un ciclo activo
        if ciclos_activos > 0:
            # Calcular eficiencia de admisión del centro usando solo solicitudes reales
            total_solicitudes_admitidas_reales = 0
            for cycle in valid_cycles:
                if center_data.get(f'{cycle}_activo', 0) == 1:
                    # Para la eficiencia del centro, usar solo solicitudes admitidas reales (sin transiciones)
                    total_solicitudes_admitidas_reales += center_data[f'{cycle}_solicitudes_admitidas']

            # Métricas agregadas del centro
            center_data.update({
                'ciclos_activos': ciclos_activos,
                'total_plazas_estimadas': total_plazas_estimadas,
                'total_matriculados': total_matriculados_año_objetivo,
                'total_max_matriculados': total_max_matriculados,
                'total_solicitudes_presentadas': total_solicitudes_presentadas,
                'total_solicitudes_admitidas': total_solicitudes_admitidas,
                'total_solicitudes_admitidas_reales': total_solicitudes_admitidas_reales,
                'tasa_ocupacion_centro': (total_matriculados_año_objetivo / total_max_matriculados) if total_max_matriculados > 0 else 0,
                'ratio_demanda_centro': (total_solicitudes_presentadas / total_plazas_estimadas) if total_plazas_estimadas > 0 else 0,
                'ratio_admision_capacidad_centro': (total_solicitudes_admitidas / total_plazas_estimadas) if total_plazas_estimadas > 0 else 0,
                'eficiencia_admision_centro': (total_solicitudes_admitidas_reales / total_solicitudes_presentadas) if total_solicitudes_presentadas > 0 else 0
            })

            center_analysis.append(center_data)

    return pd.DataFrame(center_analysis)

# Ejecutar análisis
centers_df = analyze_center_occupancy(data, cycle_config)
centers_activos = centers_df[centers_df['ciclos_activos'] > 0]  # Solo centros con actividad
print(f"✅ Análisis completado para {len(centers_activos)} centros educativos activos")


🎯 Analizando ocupación actual para año: 2022-2023
   • Ciclos incluidos: ['infantil_i_ciclo', 'infantil_ii_ciclo', 'primaria', 'eso']
   • Registros de admisiones (filtrados): 34,956
   • Registros de matrícula (filtrados): 12,026
   • Años disponibles: ['2020-2021', '2021-2022', '2022-2023', '2023-2024', '2024-2025']
   • Centros con datos en ciclos válidos: 1465
✅ Análisis completado para 1464 centros educativos activos


In [5]:
# === ESTADÍSTICAS GENERALES DE OCUPACIÓN ===
print("📊 ESTADÍSTICAS DE OCUPACIÓN - SOLICITUDES VS CAPACIDAD")
print("="*65)

# Estadísticas principales
stats_ocupacion = pd.DataFrame({
    'Métrica': [
        'Centros Educativos Activos',
        'Total Plazas Estimadas',
        'Total Estudiantes Matriculados',
        'Total Estudiantes Max Matriculados',
        'Total Solicitudes Presentadas',
        'Total Solicitudes Admitidas',
        'Tasa Ocupación Promedio (%)',
        'Ratio Demanda/Capacidad Promedio',
        'Ratio Admisión/Capacidad Promedio',
        'Eficiencia Admisión Promedio (%)',
        'Centros Sobredemandados (>1.5x)',
        'Centros Subocupados (<50%)'
    ],
    'Valor': [
        len(centers_activos),
        centers_activos['total_plazas_estimadas'].sum(),
        centers_activos['total_matriculados'].sum(),
        centers_activos['total_max_matriculados'].sum(),
        centers_activos['total_solicitudes_presentadas'].sum(),
        centers_activos['total_solicitudes_admitidas'].sum(),
        centers_activos['tasa_ocupacion_centro'].mean() * 100,
        centers_activos['ratio_demanda_centro'].mean(),
        centers_activos['ratio_admision_capacidad_centro'].mean(),
        centers_activos['eficiencia_admision_centro'].mean() * 100,
        (centers_activos['ratio_demanda_centro'] > 1.5).sum(),
        (centers_activos['tasa_ocupacion_centro'] < 0.5).sum()
    ]
})

display(stats_ocupacion.round(2))


📊 ESTADÍSTICAS DE OCUPACIÓN - SOLICITUDES VS CAPACIDAD


,Métrica,Valor
0,Centros Educativos Activos,1464.00
1,Total Plazas Estimadas,131610.08
2,Total Estudiantes Matriculados,516659.00
3,Total Estudiantes Max Matriculados,560364.00
4,Total Solicitudes Presentadas,123055.00
5,Total Solicitudes Admitidas,125369.00
6,Tasa Ocupación Promedio (%),91.53
7,Ratio Demanda/Capacidad Promedio,1.32
8,Ratio Admisión/Capacidad Promedio,1.24
9,Eficiencia Admisión Promedio (%),80.02


In [6]:
# === CENTROS CON MAYOR PRESIÓN DE DEMANDA ===
print("\n🔥 CENTROS CON MAYOR PRESIÓN DE DEMANDA (Top 15)")
print("Ratio Demanda/Capacidad más alto")
print("="*70)

top_demanda = centers_activos.nlargest(15, 'ratio_demanda_centro')[
    ['center_name', 'ciclos_activos', 'total_solicitudes_presentadas', 'total_plazas_estimadas',
     'ratio_demanda_centro', 'eficiencia_admision_centro']
].copy()

top_demanda.columns = ['Centro', 'Ciclos', 'Solicitudes', 'Plazas Est.', 'Ratio Demanda', 'Eficiencia %']
top_demanda['Ratio Demanda'] = top_demanda['Ratio Demanda'].round(2)
top_demanda['Eficiencia %'] = (top_demanda['Eficiencia %'] * 100).round(1)

display(top_demanda)



🔥 CENTROS CON MAYOR PRESIÓN DE DEMANDA (Top 15)
Ratio Demanda/Capacidad más alto


,Centro,Ciclos,Solicitudes,Plazas Est.,Ratio Demanda,Eficiencia %
584,El Tren de la Fresa,1,474,38.333333,12.37,25.3
554,El Pilar,1,385,38.333333,10.04,29.9
516,Rosa,1,223,24.000000,9.29,30.5
523,Vallehermoso,1,224,29.333333,7.64,35.3
126,Puerta del Ángel,1,279,38.000000,7.34,40.5
597,Arcoiris,1,274,38.333333,7.15,40.9
696,Rayuela,1,400,61.000000,6.56,44.5
1377,Rocio Durcal,1,387,59.666667,6.49,47.0
1463,Shantala,1,247,38.333333,6.44,50.2
141,La Marazuela,1,162,25.333333,6.39,42.0


In [7]:
# === CENTROS CON MAYOR OCUPACIÓN ===
print("\n📈 CENTROS CON MAYOR OCUPACIÓN (Top 15)")
print("Tasa de ocupación más alta")
print("="*70)

top_ocupacion = centers_activos.nlargest(15, 'tasa_ocupacion_centro')[
    ['center_name', 'ciclos_activos', 'total_matriculados', 'total_max_matriculados', 'total_plazas_estimadas',
     'tasa_ocupacion_centro', 'ratio_demanda_centro']
].copy()

top_ocupacion.columns = ['Centro', 'Ciclos', 'Matriculados', 'Total Max Matriculados', 'Plazas Est.', 'Ocupación %', 'Ratio Demanda']
top_ocupacion['Ocupación %'] = (top_ocupacion['Ocupación %'] * 100).round(1)
top_ocupacion['Ratio Demanda'] = top_ocupacion['Ratio Demanda'].round(2)

display(top_ocupacion)



📈 CENTROS CON MAYOR OCUPACIÓN (Top 15)
Tasa de ocupación más alta


,Centro,Ciclos,Matriculados,Total Max Matriculados,Plazas Est.,Ocupación %,Ratio Demanda
6,Gustavo Adolfo Bécquer,1,511,511,127.750000,100.0,1.14
21,Casa de los Niños,2,126,126,42.000000,100.0,4.24
24,Cardenal Cisneros,1,354,354,88.500000,100.0,1.21
40,Victoria Kent,1,387,387,96.750000,100.0,1.05
52,Villablanca,1,843,843,210.750000,100.0,0.97
58,Federico García Lorca,1,220,220,55.000000,100.0,1.20
63,Colorines,1,52,52,17.333333,100.0,0.00
67,Pimpollitos,1,116,116,38.666667,100.0,3.23
77,Antusana,1,34,34,11.333333,100.0,3.71
78,República del Uruguay,2,467,467,100.500000,100.0,0.57


In [8]:
# === ANÁLISIS POR CICLOS EDUCATIVOS ===
print("\n📚 ANÁLISIS POR CICLOS - OCUPACIÓN Y DEMANDA")
print("="*55)

cycle_summary = []
for cycle, config in cycle_config.items():
    # Centros que ofrecen este ciclo
    cycle_centros = centers_activos[centers_activos[f'{cycle}_activo'] == 1]

    if len(cycle_centros) > 0:
        # Métricas agregadas del ciclo usando columnas corregidas
        total_max_matriculados = cycle_centros[f'{cycle}_max_matriculados'].sum()
        total_matriculados_objetivo = cycle_centros[f'{cycle}_matriculados_objetivo'].sum()
        total_plazas = cycle_centros[f'{cycle}_plazas_estimadas'].sum()
        total_solicitudes = cycle_centros[f'{cycle}_solicitudes_presentadas'].sum()
        total_admitidas = cycle_centros[f'{cycle}_solicitudes_admitidas_corregidas'].sum()

        # Transiciones para primaria
        total_transiciones = 0
        if cycle == 'primaria':
            total_transiciones = cycle_centros[f'{cycle}_transiciones_infantil'].sum()

        # Calcular promedios
        tasa_ocupacion_promedio = cycle_centros[f'{cycle}_tasa_ocupacion'].mean()
        ratio_demanda_promedio = cycle_centros[f'{cycle}_ratio_demanda'].mean()
        ratio_admision_promedio = cycle_centros[f'{cycle}_ratio_admision_capacidad'].mean()
        eficiencia_promedio = cycle_centros[f'{cycle}_eficiencia_admision'].mean()

        cycle_summary.append({
            'Ciclo': config['name'],
            'Centros': len(cycle_centros),
            'Max Matriculados': total_max_matriculados,
            'Matriculados 2022-23': total_matriculados_objetivo,
            'Plazas Est.': total_plazas,
            'Solicitudes': total_solicitudes,
            'Admitidas': total_admitidas,
            'Transiciones': total_transiciones,
            'Ocupación %': tasa_ocupacion_promedio * 100,
            'Ratio Demanda': ratio_demanda_promedio,
            'Ratio Admisión': ratio_admision_promedio,
            'Eficiencia %': eficiencia_promedio * 100
        })

cycle_summary_df = pd.DataFrame(cycle_summary)
display(cycle_summary_df.round(1))



📚 ANÁLISIS POR CICLOS - OCUPACIÓN Y DEMANDA


,Ciclo,Centros,Max Matriculados,Matriculados 2022-23,Plazas Est.,Solicitudes,Admitidas,Transiciones,Ocupación %,Ratio Demanda,Ratio Admisión,Eficiencia %
0,Infantil I,426,35749,31332,11916.3,41750,30406.0,0.0,82.8,3.3,2.4,70.3
1,Infantil II,827,107083,94437,35694.3,30363,26641.0,0.0,87.5,0.8,0.7,89.5
2,Primaria,811,244603,229423,40767.2,8758,34947.0,31235.0,93.4,0.2,0.9,51.0
3,ESO,352,172929,161467,43232.2,42184,33375.0,0.0,90.7,0.9,0.7,76.4


In [9]:
# === EJEMPLOS DE CÁLCULO DE PLAZAS ESTIMADAS ===
print("\n🔍 EJEMPLOS DE CÁLCULO DE PLAZAS ESTIMADAS (Top 5 por plazas)")
print("Usando máximo de (matriculados + admitidos) ÷ años entre todos los años")
print("="*80)

# Seleccionar los 5 centros con más plazas estimadas para mostrar ejemplos
top_capacity_centers = centers_activos.nlargest(5, 'total_plazas_estimadas')

ejemplos_calculo = []
for _, center in top_capacity_centers.iterrows():
    center_ejemplo = {
        'Centro': center['center_name'],
        'Total Plazas Est.': center['total_plazas_estimadas']
    }

    # Mostrar cálculo para cada ciclo activo
    detalles_ciclos = []
    for cycle, config in cycle_config.items():
        if center[f'{cycle}_activo'] == 1:
            max_mat = center[f'{cycle}_max_matriculados']
            max_adm = center[f'{cycle}_max_admitidos']
            max_total = center[f'{cycle}_max_total_estudiantes']
            año_max = center[f'{cycle}_año_max']
            plazas = center[f'{cycle}_plazas_estimadas']
            años_perm = config['años_permanencia']

            detalle = f"{config['name']}: ({max_mat} mat. + {max_adm} adm.) = {max_total} ({año_max}) ÷ {años_perm} años = {plazas:.1f} plazas"
            detalles_ciclos.append(detalle)

    center_ejemplo['Cálculo Detallado'] = ' | '.join(detalles_ciclos)
    ejemplos_calculo.append(center_ejemplo)

ejemplos_df = pd.DataFrame(ejemplos_calculo)
for _, ejemplo in ejemplos_df.iterrows():
    print(f"\n🏫 {ejemplo['Centro']}")
    print(f"   Total Plazas Estimadas: {ejemplo['Total Plazas Est.']:.1f}")
    print(f"   Cálculo: {ejemplo['Cálculo Detallado']}")



🔍 EJEMPLOS DE CÁLCULO DE PLAZAS ESTIMADAS (Top 5 por plazas)
Usando máximo de (matriculados + admitidos) ÷ años entre todos los años

🏫 Joaquín Costa
   Total Plazas Estimadas: 294.0
   Cálculo: Infantil II: (435 mat. + 150 adm.) = 435 (2024-2025) ÷ 3 años = 145.0 plazas | Primaria: (894 mat. + 1 adm.) = 894 (2020-2021) ÷ 6 años = 149.0 plazas

🏫 La Luna
   Total Plazas Estimadas: 277.0
   Cálculo: Infantil II: (275 mat. + 75 adm.) = 275 (2020-2021) ÷ 3 años = 91.7 plazas | Primaria: (491 mat. + 0 adm.) = 491 (2024-2025) ÷ 6 años = 81.8 plazas | ESO: (414 mat. + 49 adm.) = 414 (2024-2025) ÷ 4 años = 103.5 plazas

🏫 El Espinillo
   Total Plazas Estimadas: 263.8
   Cálculo: ESO: (1055 mat. + 191 adm.) = 1055 (2023-2024) ÷ 4 años = 263.8 plazas

🏫 Maestro Rodrigo
   Total Plazas Estimadas: 262.1
   Cálculo: Infantil II: (230 mat. + 75 adm.) = 230 (2020-2021) ÷ 3 años = 76.7 plazas | Primaria: (562 mat. + 5 adm.) = 562 (2024-2025) ÷ 6 años = 93.7 plazas | ESO: (367 mat. + 15 adm.) = 367 (

In [10]:
# === CENTROS CON CORRECCIÓN DE PRIMARIA ===
print("\n🔄 CENTROS CON TRANSICIONES AUTOMÁTICAS INFANTIL II → PRIMARIA")
print("="*65)

centros_con_transiciones = centers_activos[
    centers_activos['primaria_transiciones_infantil'] > 0
].copy()

if len(centros_con_transiciones) > 0:
    transiciones_detalle = centros_con_transiciones[
        ['center_name', 'primaria_matriculados_objetivo', 'primaria_solicitudes_admitidas',
         'primaria_transiciones_infantil', 'primaria_solicitudes_admitidas_corregidas']
    ].copy()

    transiciones_detalle.columns = [
        'Centro', 'Matriculados Primaria', 'Admisiones Solicitudes',
        'Transiciones Infantil II', 'Total Admisiones Corregidas'
    ]

    # Mostrar los primeros 10
    display(transiciones_detalle.head(10).round(1))

    print(f"\n📊 Resumen de transiciones:")
    print(f"   • Centros con transiciones: {len(centros_con_transiciones)}")
    print(f"   • Total transiciones automáticas: {centros_con_transiciones['primaria_transiciones_infantil'].sum():.1f}")
    print(f"   • Promedio transiciones por centro: {centros_con_transiciones['primaria_transiciones_infantil'].mean():.1f}")
else:
    print("No se encontraron centros con transiciones automáticas")



🔄 CENTROS CON TRANSICIONES AUTOMÁTICAS INFANTIL II → PRIMARIA


,Centro,Matriculados Primaria,Admisiones Solicitudes,Transiciones Infantil II,Total Admisiones Corregidas
0,Maestro Padilla,594,6,81.7,87.7
1,La Zarzuela,419,5,53.7,58.7
2,León Felipe,92,3,9.7,12.7
3,Santa Teresa,312,1,31.3,32.3
4,José Bergamín,77,2,8.7,10.7
7,Padre Garralda,287,22,47.0,69.0
9,El Prado,529,7,55.7,62.7
12,Ciudad de Columbia,300,1,47.0,48.0
14,Francisco de Goya,143,5,21.3,26.3
15,Isaac Peral,310,10,31.7,41.7



📊 Resumen de transiciones:
   • Centros con transiciones: 825
   • Total transiciones automáticas: 31479.0
   • Promedio transiciones por centro: 38.2


In [11]:
# === ANÁLISIS DE EFICIENCIA POR CENTRO ===
print("\n⚡ ANÁLISIS DE EFICIENCIA - ADMISIÓN VS CAPACIDAD")
print("="*60)

# Clasificar centros por eficiencia
def clasificar_eficiencia(ratio_admision):
    if ratio_admision >= 1.0:
        return "Sobrecapacidad"
    elif ratio_admision >= 0.8:
        return "Alta eficiencia"
    elif ratio_admision >= 0.5:
        return "Eficiencia media"
    else:
        return "Baja eficiencia"

centers_activos['clasificacion_eficiencia'] = centers_activos['ratio_admision_capacidad_centro'].apply(clasificar_eficiencia)

eficiencia_resumen = centers_activos['clasificacion_eficiencia'].value_counts()
eficiencia_porcentajes = (eficiencia_resumen / len(centers_activos) * 100).round(1)

eficiencia_stats = pd.DataFrame({
    'Clasificación': eficiencia_resumen.index,
    'Centros': eficiencia_resumen.values,
    'Porcentaje': eficiencia_porcentajes.values
})

print("Distribución de eficiencia de centros:")
display(eficiencia_stats)

print("\n✅ ANÁLISIS DE OCUPACIÓN EDUCATIVA COMPLETADO")
print("="*60)



⚡ ANÁLISIS DE EFICIENCIA - ADMISIÓN VS CAPACIDAD
Distribución de eficiencia de centros:


,Clasificación,Centros,Porcentaje
0,Eficiencia media,495,33.8
1,Alta eficiencia,481,32.9
2,Sobrecapacidad,434,29.6
3,Baja eficiencia,54,3.7



✅ ANÁLISIS DE OCUPACIÓN EDUCATIVA COMPLETADO


In [12]:
# === ANÁLISIS DE COBERTURA EDUCATIVA POR MUNICIPIO ===
print("\n🏛️ ANÁLISIS DE COBERTURA EDUCATIVA MUNICIPAL")
print("Validación de capacidad vs necesidades demográficas")
print("="*70)

def load_demographic_data():
    """Carga datos demográficos para el análisis de cobertura"""
    demographics = pd.read_csv('../downloads/normalizacion/municipality_demographics.csv')

    # Mapeo de rangos de edad a ciclos educativos
    age_to_cycle_mapping = {
        '0-4': ['infantil_i_ciclo', 'infantil_ii_ciclo'],  # 0-2 y 3-5
        '5-9': ['infantil_ii_ciclo', 'primaria'],         # 3-5 y 6-11
        '10-14': ['primaria', 'eso'],                      # 6-11 y 12-15
        '15-19': ['eso']                                   # 12-15
    }

    return demographics, age_to_cycle_mapping

def analyze_municipal_coverage(data, centers_df, cycle_config):
    """Analiza la cobertura educativa por municipio"""

    # Cargar datos demográficos
    demographics, age_mapping = load_demographic_data()

    # Obtener relación centro-municipio
    edu_municipality = data['edu_municipality']
    municipalities = data['municipality']

    # Usar el año más reciente disponible para demografía
    latest_year = demographics['year'].max()
    demographics_latest = demographics[demographics['year'] == latest_year].copy()

    print(f"📊 Usando datos demográficos del año: {latest_year}")
    print(f"   • Municipios con datos demográficos: {demographics_latest['id_secondary_municipality'].nunique()}")

    # Análisis por municipio
    municipal_analysis = []

    # Obtener municipios únicos que tienen centros educativos
    municipios_con_centros = edu_municipality['id_municipality'].unique()

    for municipality_id in municipios_con_centros:
        # Información del municipio
        muni_info = municipalities[municipalities['id'] == municipality_id]
        if muni_info.empty:
            continue

        muni_info = muni_info.iloc[0]
        muni_secondary_id = muni_info['id_secondary']

        # Obtener datos demográficos del municipio
        muni_demographics = demographics_latest[
            demographics_latest['id_secondary_municipality'] == muni_secondary_id
        ].copy()

        if muni_demographics.empty:
            continue  # Saltar municipios sin datos demográficos

        # Obtener centros educativos del municipio
        centros_municipio = edu_municipality[
            edu_municipality['id_municipality'] == municipality_id
        ]['id_education'].unique()

        # Filtrar centros activos en nuestro análisis
        centros_activos_municipio = centers_df[
            centers_df['id_education'].isin(centros_municipio)
        ].copy()

        if centros_activos_municipio.empty:
            continue  # Saltar municipios sin centros activos

        # Inicializar datos del municipio
        municipal_data = {
            'id_municipality': municipality_id,
            'municipality_name': muni_info['name'],
            'total_centers': len(centros_activos_municipio),
            'total_population_0_19': 0
        }

        # Calcular población total por grupos de edad relevantes
        for _, demo_row in muni_demographics.iterrows():
            age_range = demo_row['range']
            population = demo_row['total']

            if age_range in ['0-4', '5-9', '10-14', '15-19']:
                municipal_data['total_population_0_19'] += population
                municipal_data[f'population_{age_range}'] = population

        # Análisis por ciclo educativo
        for cycle in ['infantil_i_ciclo', 'infantil_ii_ciclo', 'primaria', 'eso']:
            config = cycle_config[cycle]

            # Usar máximo de matrícula histórica en lugar de plazas estimadas
            cycle_max_matriculados = centros_activos_municipio[f'{cycle}_max_matriculados'].sum()
            cycle_plazas_estimadas = centros_activos_municipio[f'{cycle}_plazas_estimadas'].sum()
            cycle_solicitudes = centros_activos_municipio[f'{cycle}_solicitudes_presentadas'].sum()
            cycle_admitidos = centros_activos_municipio[f'{cycle}_solicitudes_admitidas_corregidas'].sum()

            # Centros que ofrecen este ciclo
            centros_con_ciclo = centros_activos_municipio[
                centros_activos_municipio[f'{cycle}_activo'] == 1
            ]
            num_centros_ciclo = len(centros_con_ciclo)

            municipal_data.update({
                f'{cycle}_capacity': cycle_max_matriculados,  # Cambiar a máximo matriculados
                f'{cycle}_plazas_estimadas': cycle_plazas_estimadas,  # Mantener para referencia
                f'{cycle}_max_matriculados': cycle_max_matriculados,
                f'{cycle}_solicitudes': cycle_solicitudes,
                f'{cycle}_admitidos': cycle_admitidos,
                f'{cycle}_num_centers': num_centros_ciclo,
                f'{cycle}_centers_names': ', '.join(centros_con_ciclo['center_name'].tolist()) if num_centros_ciclo > 0 else 'Ninguno'
            })

        # Calcular estimaciones de necesidad educativa por ciclo
        # Basado en población por edad y años de permanencia en cada ciclo

        # Infantil I (0-2 años): aproximadamente 3/5 de la población 0-4
        pop_0_4 = municipal_data.get('population_0-4', 0)
        estimated_need_infantil_i = (pop_0_4 * 3) / 5  # 3 años de los 5 años del rango 0-4

        # Infantil II (3-5 años): aproximadamente 2/5 de la población 0-4 + 1/5 de la población 5-9
        pop_5_9 = municipal_data.get('population_5-9', 0)
        estimated_need_infantil_ii = (pop_0_4 * 2) / 5 + (pop_5_9 * 1) / 5

        # Primaria (6-11 años): aproximadamente 4/5 de la población 5-9 + 2/5 de la población 10-14
        pop_10_14 = municipal_data.get('population_10-14', 0)
        estimated_need_primaria = (pop_5_9 * 4) / 5 + (pop_10_14 * 2) / 5

        # ESO (12-15 años): aproximadamente 3/5 de la población 10-14 + 1/5 de la población 15-19
        pop_15_19 = municipal_data.get('population_15-19', 0)
        estimated_need_eso = (pop_10_14 * 3) / 5 + (pop_15_19 * 1) / 5

        # Guardar estimaciones de necesidad
        needs = {
            'infantil_i_ciclo': estimated_need_infantil_i,
            'infantil_ii_ciclo': estimated_need_infantil_ii,
            'primaria': estimated_need_primaria,
            'eso': estimated_need_eso
        }

        # Calcular indicadores de cobertura por ciclo
        total_estimated_need = 0
        total_capacity = 0
        cycles_with_need = 0
        cycles_covered = 0

        for cycle, estimated_need in needs.items():
            capacity = municipal_data[f'{cycle}_capacity']

            # Ratio de cobertura (capacidad / necesidad estimada)
            coverage_ratio = (capacity / estimated_need) if estimated_need > 0 else float('inf') if capacity > 0 else 0

            # ¿Está cubierto este ciclo? (ratio >= 0.9, permitiendo 10% de margen)
            is_covered = coverage_ratio >= 1 if estimated_need > 0 else capacity > 0

            municipal_data.update({
                f'{cycle}_estimated_need': estimated_need,
                f'{cycle}_coverage_ratio': coverage_ratio,
                f'{cycle}_is_covered': is_covered,
                f'{cycle}_deficit': max(0, estimated_need - capacity)
            })

            if estimated_need > 0:
                cycles_with_need += 1
                total_estimated_need += estimated_need
                total_capacity += capacity
                if is_covered:
                    cycles_covered += 1

        # Métricas agregadas del municipio
        overall_coverage_ratio = (total_capacity / total_estimated_need) if total_estimated_need > 0 else 0
        coverage_percentage = (cycles_covered / cycles_with_need * 100) if cycles_with_need > 0 else 0

        # NUEVO CÁLCULO: Acceso a ciclos educativos (independiente de capacidad vs población)
        cycles_with_access = 0  # Ciclos a los que el municipio tiene acceso físico
        cycles_available = ['infantil_i_ciclo', 'infantil_ii_ciclo', 'primaria', 'eso']

        for cycle in cycles_available:
            # Si el municipio tiene al menos un centro que ofrece este ciclo
            if municipal_data[f'{cycle}_num_centers'] > 0:
                cycles_with_access += 1

        # Porcentaje de acceso: ciclos disponibles vs total de ciclos posibles
        access_percentage = (cycles_with_access / len(cycles_available)) * 100

        # Clasificación de acceso educativo
        if cycles_with_access == len(cycles_available):
            access_classification = "Acceso Completo"
        elif cycles_with_access >= 3:
            access_classification = "Acceso Bueno"
        elif cycles_with_access >= 2:
            access_classification = "Acceso Parcial"
        elif cycles_with_access >= 1:
            access_classification = "Acceso Limitado"
        else:
            access_classification = "Sin Acceso"

        # ¿Tiene el municipio educación básica completa? (Infantil II + Primaria + ESO)
        has_basic_education = (
            municipal_data['infantil_ii_ciclo_num_centers'] > 0 and
            municipal_data['primaria_num_centers'] > 0 and
            municipal_data['eso_num_centers'] > 0
        )

        # ¿Tiene el municipio educación obligatoria? (Primaria + ESO)
        has_mandatory_education = (
            municipal_data['primaria_num_centers'] > 0 and
            municipal_data['eso_num_centers'] > 0
        )

        municipal_data.update({
            'total_estimated_need': total_estimated_need,
            'total_capacity': total_capacity,
            'overall_coverage_ratio': overall_coverage_ratio,
            'cycles_with_need': cycles_with_need,
            'cycles_covered': cycles_covered,
            'coverage_percentage': coverage_percentage,
            'is_fully_covered': cycles_covered == cycles_with_need and cycles_with_need > 0,
            'total_deficit': max(0, total_estimated_need - total_capacity),
            # NUEVAS MÉTRICAS DE ACCESO
            'cycles_with_access': cycles_with_access,
            'access_percentage': access_percentage,
            'access_classification': access_classification,
            'has_basic_education': has_basic_education,
            'has_mandatory_education': has_mandatory_education,
            'missing_cycles': [cycle for cycle in cycles_available if municipal_data[f'{cycle}_num_centers'] == 0]
        })

        municipal_analysis.append(municipal_data)

    return pd.DataFrame(municipal_analysis)

# Ejecutar análisis de cobertura municipal
print("⏳ Procesando análisis de cobertura municipal...")
municipal_coverage_df = analyze_municipal_coverage(data, centers_df, cycle_config)

print(f"✅ Análisis completado para {len(municipal_coverage_df)} municipios")
print(f"   • Municipios con cobertura completa: {municipal_coverage_df['is_fully_covered'].sum()}")
print(f"   • Municipios con déficit: {(municipal_coverage_df['total_deficit'] > 0).sum()}")



🏛️ ANÁLISIS DE COBERTURA EDUCATIVA MUNICIPAL
Validación de capacidad vs necesidades demográficas
⏳ Procesando análisis de cobertura municipal...
📊 Usando datos demográficos del año: 2024
   • Municipios con datos demográficos: 179
✅ Análisis completado para 179 municipios
   • Municipios con cobertura completa: 46
   • Municipios con déficit: 63


In [13]:
# === ESTADÍSTICAS DE COBERTURA MUNICIPAL ===
print("\n📈 ESTADÍSTICAS DE COBERTURA EDUCATIVA MUNICIPAL")
print("="*65)

# Estadísticas generales
coverage_stats = pd.DataFrame({
    'Métrica': [
        'Municipios Analizados',
        'Población Total 0-19 años',
        'Necesidad Educativa Estimada',
        'Capacidad Total Disponible',
        'Ratio Cobertura Promedio',
        'Municipios con Cobertura Completa',
        'Municipios con Déficit Educativo',
        'Déficit Total Estimado (plazas)',
        'Cobertura Promedio (%)',
        'Municipios sin Infantil I',
        'Municipios sin ESO'
    ],
    'Valor': [
        len(municipal_coverage_df),
        municipal_coverage_df['total_population_0_19'].sum(),
        municipal_coverage_df['total_estimated_need'].sum(),
        municipal_coverage_df['total_capacity'].sum(),
        municipal_coverage_df['overall_coverage_ratio'].mean(),
        municipal_coverage_df['is_fully_covered'].sum(),
        (municipal_coverage_df['total_deficit'] > 0).sum(),
        municipal_coverage_df['total_deficit'].sum(),
        municipal_coverage_df['coverage_percentage'].mean(),
        (municipal_coverage_df['infantil_i_ciclo_num_centers'] == 0).sum(),
        (municipal_coverage_df['eso_num_centers'] == 0).sum()
    ]
})

display(coverage_stats.round(2))



📈 ESTADÍSTICAS DE COBERTURA EDUCATIVA MUNICIPAL


,Métrica,Valor
0,Municipios Analizados,179.00
1,Población Total 0-19 años,1336662.00
2,Necesidad Educativa Estimada,1031788.40
3,Capacidad Total Disponible,773326.80
4,Ratio Cobertura Promedio,12.12
5,Municipios con Cobertura Completa,46.00
6,Municipios con Déficit Educativo,63.00
7,Déficit Total Estimado (plazas),365758.60
8,Cobertura Promedio (%),58.52
9,Municipios sin Infantil I,24.00


In [14]:
# === MUNICIPIOS CON MAYOR DÉFICIT EDUCATIVO ===
print("\n🚨 MUNICIPIOS CON MAYOR DÉFICIT EDUCATIVO (Top 15)")
print("Necesidades no cubiertas por la capacidad actual")
print("="*70)

deficit_municipios = municipal_coverage_df[municipal_coverage_df['total_deficit'] > 0].copy()
top_deficit = deficit_municipios.nlargest(15, 'total_deficit')[
    ['municipality_name', 'total_population_0_19', 'total_estimated_need',
     'total_capacity', 'total_deficit', 'coverage_percentage', 'cycles_covered', 'cycles_with_need']
].copy()

top_deficit['deficit_percentage'] = (top_deficit['total_deficit'] / top_deficit['total_estimated_need'] * 100).round(1)

print("Columnas: Población 0-19, Necesidad Estimada, Capacidad, Déficit, % Cobertura, Ciclos Cubiertos/Total")
display(top_deficit.round(1))



🚨 MUNICIPIOS CON MAYOR DÉFICIT EDUCATIVO (Top 15)
Necesidades no cubiertas por la capacidad actual
Columnas: Población 0-19, Necesidad Estimada, Capacidad, Déficit, % Cobertura, Ciclos Cubiertos/Total


,municipality_name,total_population_0_19,total_estimated_need,total_capacity,total_deficit,coverage_percentage,cycles_covered,cycles_with_need,deficit_percentage
168,Madrid,574787,446539.0,193740.0,252799.0,0.0,0,4,56.6
155,Alcalá de Henares,37603,28267.8,17269.2,10998.6,0.0,0,4,38.9
173,Pozuelo de Alarcón,19757,14750.6,4856.4,9894.2,0.0,0,4,67.1
161,Boadilla del Monte,18226,13534.8,5301.6,8233.2,0.0,0,4,60.8
175,Las Rozas de Madrid,23532,17757.6,9730.8,8026.8,0.0,0,4,45.2
156,Alcobendas,25756,19860.0,12560.4,7299.6,0.0,0,4,36.8
169,Majadahonda,16408,12276.0,6074.4,6201.6,0.0,0,4,50.5
157,Alcorcón,32215,24433.4,18846.0,5587.4,0.0,0,4,22.9
176,San Sebastián de los Reyes,20643,16335.8,12363.6,3972.2,25.0,1,4,24.3
178,Valdemoro,20248,15272.0,11448.0,3824.0,0.0,0,4,25.0


In [15]:
# === ANÁLISIS POR CICLO EDUCATIVO ===
print("\n📚 ANÁLISIS DE COBERTURA POR CICLO EDUCATIVO")
print("="*55)

cycles_analysis = []
for cycle in ['infantil_i_ciclo', 'infantil_ii_ciclo', 'primaria', 'eso']:
    config = cycle_config[cycle]

    # Municipios que tienen este ciclo
    municipios_con_ciclo = municipal_coverage_df[
        municipal_coverage_df[f'{cycle}_num_centers'] > 0
    ]

    # Estadísticas del ciclo
    total_need = municipal_coverage_df[f'{cycle}_estimated_need'].sum()
    total_capacity = municipal_coverage_df[f'{cycle}_capacity'].sum()
    municipios_cubiertos = municipal_coverage_df[f'{cycle}_is_covered'].sum()
    deficit_total = municipal_coverage_df[f'{cycle}_deficit'].sum()

    cycles_analysis.append({
        'Ciclo': config['name'],
        'Municipios con Oferta': len(municipios_con_ciclo),
        'Necesidad Total': total_need,
        'Capacidad Total': total_capacity,
        'Municipios Cubiertos': municipios_cubiertos,
        'Déficit Total': deficit_total,
        'Ratio Cobertura': total_capacity / total_need if total_need > 0 else 0,
        '% Municipios Cubiertos': municipios_cubiertos / len(municipal_coverage_df) * 100
    })

cycles_df = pd.DataFrame(cycles_analysis)
display(cycles_df.round(2))



📚 ANÁLISIS DE COBERTURA POR CICLO EDUCATIVO


,Ciclo,Municipios con Oferta,Necesidad Total,Capacidad Total,Municipios Cubiertos,Déficit Total,Ratio Cobertura,% Municipios Cubiertos
0,Infantil I,155,158213.4,38459,56,113805.6,0.24,31.28
1,Infantil II,179,170823.0,116773,126,48348.2,0.68,70.39
2,Primaria,179,407447.2,267262,111,120282.0,0.66,62.01
3,ESO,179,295304.8,222069,126,97924.0,0.75,70.39


In [16]:
# === MUNICIPIOS SIN OFERTA EDUCATIVA POR CICLO ===
print("\n❌ MUNICIPIOS SIN OFERTA EDUCATIVA POR CICLO")
print("="*50)

for cycle in ['infantil_i_ciclo', 'infantil_ii_ciclo', 'primaria', 'eso']:
    config = cycle_config[cycle]

    municipios_sin_oferta = municipal_coverage_df[
        (municipal_coverage_df[f'{cycle}_num_centers'] == 0) &
        (municipal_coverage_df[f'{cycle}_estimated_need'] > 0)
    ].copy()

    if len(municipios_sin_oferta) > 0:
        print(f"\n🔴 {config['name']} - {len(municipios_sin_oferta)} municipios sin oferta:")
        top_sin_oferta = municipios_sin_oferta.nlargest(10, f'{cycle}_estimated_need')[
            ['municipality_name', f'{cycle}_estimated_need', 'total_centers']
        ]
        print("   Top 10 por necesidad estimada:")
        for _, row in top_sin_oferta.iterrows():
            print(f"   • {row['municipality_name']}: {row[f'{cycle}_estimated_need']:.0f} estudiantes estimados, {row['total_centers']} centros totales")



❌ MUNICIPIOS SIN OFERTA EDUCATIVA POR CICLO

🔴 Infantil I - 24 municipios sin oferta:
   Top 10 por necesidad estimada:
   • Moralzarzal: 298 estudiantes estimados, 3 centros totales
   • Morata de Tajuña: 191 estudiantes estimados, 2 centros totales
   • Talamanca de Jarama: 117 estudiantes estimados, 1 centros totales
   • Valdetorres de Jarama: 107 estudiantes estimados, 1 centros totales
   • Quijorna: 94 estudiantes estimados, 4 centros totales
   • Valdilecha: 89 estudiantes estimados, 1 centros totales
   • Navas del Rey: 89 estudiantes estimados, 1 centros totales
   • Villaconejos: 83 estudiantes estimados, 3 centros totales
   • Pelayos de la Presa: 82 estudiantes estimados, 2 centros totales
   • Navacerrada: 69 estudiantes estimados, 3 centros totales


In [17]:
# === RESUMEN EJECUTIVO DE COBERTURA ===
print("\n🎯 RESUMEN EJECUTIVO - COBERTURA EDUCATIVA MUNICIPAL")
print("="*65)

# Clasificación de municipios por nivel de cobertura
excelente = municipal_coverage_df[municipal_coverage_df['coverage_percentage'] >= 100]
buena = municipal_coverage_df[(municipal_coverage_df['coverage_percentage'] >= 75) & (municipal_coverage_df['coverage_percentage'] < 100)]
regular = municipal_coverage_df[(municipal_coverage_df['coverage_percentage'] >= 50) & (municipal_coverage_df['coverage_percentage'] < 75)]
deficiente = municipal_coverage_df[municipal_coverage_df['coverage_percentage'] < 50]

print(f"📊 CLASIFICACIÓN POR NIVEL DE COBERTURA:")
print(f"   🟢 Excelente (100%): {len(excelente)} municipios")
print(f"   🟡 Buena (75-99%): {len(buena)} municipios")
print(f"   🟠 Regular (50-74%): {len(regular)} municipios")
print(f"   🔴 Deficiente (<50%): {len(deficiente)} municipios")

print(f"\n🎯 INDICADORES CLAVE:")
print(f"   • Cobertura promedio regional: {municipal_coverage_df['coverage_percentage'].mean():.1f}%")
print(f"   • Ratio capacidad/necesidad: {municipal_coverage_df['overall_coverage_ratio'].mean():.2f}")
print(f"   • Déficit educativo total: {municipal_coverage_df['total_deficit'].sum():.0f} plazas")
print(f"   • Población estudiantil analizada: {municipal_coverage_df['total_population_0_19'].sum():.0f} personas")

# Municipios críticos que requieren atención inmediata
criticos = municipal_coverage_df[
    (municipal_coverage_df['coverage_percentage'] < 50) &
    (municipal_coverage_df['total_population_0_19'] > 100)
]

if len(criticos) > 0:
    print(f"\n🚨 MUNICIPIOS CRÍTICOS (>100 hab. y <50% cobertura): {len(criticos)}")
    for _, municipio in criticos.head(5).iterrows():
        print(f"   • {municipio['municipality_name']}: {municipio['coverage_percentage']:.1f}% cobertura, {municipio['total_population_0_19']:.0f} hab. 0-19 años")

print(f"\n✅ Análisis de cobertura educativa municipal completado")
print(f"💡 Los municipios pueden usar estos datos para planificar expansión educativa")



🎯 RESUMEN EJECUTIVO - COBERTURA EDUCATIVA MUNICIPAL
📊 CLASIFICACIÓN POR NIVEL DE COBERTURA:
   🟢 Excelente (100%): 46 municipios
   🟡 Buena (75-99%): 63 municipios
   🟠 Regular (50-74%): 12 municipios
   🔴 Deficiente (<50%): 58 municipios

🎯 INDICADORES CLAVE:
   • Cobertura promedio regional: 58.5%
   • Ratio capacidad/necesidad: 12.12
   • Déficit educativo total: 365759 plazas
   • Población estudiantil analizada: 1336662 personas

🚨 MUNICIPIOS CRÍTICOS (>100 hab. y <50% cobertura): 58
   • Daganzo de Arriba: 25.0% cobertura, 2800 hab. 0-19 años
   • Colmenar de Oreja: 25.0% cobertura, 1700 hab. 0-19 años
   • Moralzarzal: 0.0% cobertura, 3339 hab. 0-19 años
   • Guadarrama: 0.0% cobertura, 3487 hab. 0-19 años
   • Hoyo de Manzanares: 25.0% cobertura, 1847 hab. 0-19 años

✅ Análisis de cobertura educativa municipal completado
💡 Los municipios pueden usar estos datos para planificar expansión educativa


In [18]:
# === ANÁLISIS DE ACCESO A CICLOS EDUCATIVOS ===
print("\n🏫 ANÁLISIS DE ACCESO A CICLOS EDUCATIVOS")
print("Evaluación independiente de disponibilidad de servicios educativos")
print("="*70)

# Estadísticas de acceso general
access_stats = pd.DataFrame({
    'Métrica de Acceso': [
        'Municipios con Acceso Completo (4/4 ciclos)',
        'Municipios con Acceso Bueno (3/4 ciclos)',
        'Municipios con Acceso Parcial (2/4 ciclos)',
        'Municipios con Acceso Limitado (1/4 ciclos)',
        'Municipios sin Acceso (0/4 ciclos)',
        'Municipios con Educación Básica Completa',
        'Municipios con Educación Obligatoria',
        'Porcentaje Acceso Promedio'
    ],
    'Valor': [
        (municipal_coverage_df['access_classification'] == 'Acceso Completo').sum(),
        (municipal_coverage_df['access_classification'] == 'Acceso Bueno').sum(),
        (municipal_coverage_df['access_classification'] == 'Acceso Parcial').sum(),
        (municipal_coverage_df['access_classification'] == 'Acceso Limitado').sum(),
        (municipal_coverage_df['access_classification'] == 'Sin Acceso').sum(),
        municipal_coverage_df['has_basic_education'].sum(),
        municipal_coverage_df['has_mandatory_education'].sum(),
        municipal_coverage_df['access_percentage'].mean()
    ]
})

display(access_stats.round(1))



🏫 ANÁLISIS DE ACCESO A CICLOS EDUCATIVOS
Evaluación independiente de disponibilidad de servicios educativos


,Métrica de Acceso,Valor
0,Municipios con Acceso Completo (4/4 ciclos),155.0
1,Municipios con Acceso Bueno (3/4 ciclos),24.0
2,Municipios con Acceso Parcial (2/4 ciclos),0.0
3,Municipios con Acceso Limitado (1/4 ciclos),0.0
4,Municipios sin Acceso (0/4 ciclos),0.0
5,Municipios con Educación Básica Completa,179.0
6,Municipios con Educación Obligatoria,179.0
7,Porcentaje Acceso Promedio,96.6


In [19]:
# === DISTRIBUCIÓN DE ACCESO POR CLASIFICACIÓN ===
print("\n📊 DISTRIBUCIÓN DE MUNICIPIOS POR NIVEL DE ACCESO")
print("="*55)

access_distribution = municipal_coverage_df['access_classification'].value_counts()
access_percentages = (access_distribution / len(municipal_coverage_df) * 100).round(1)

access_summary = pd.DataFrame({
    'Clasificación de Acceso': access_distribution.index,
    'Municipios': access_distribution.values,
    'Porcentaje': access_percentages.values
})

print("Distribución por nivel de acceso a ciclos educativos:")
display(access_summary)

# Estadísticas detalladas por nivel de acceso
print(f"\n📈 ESTADÍSTICAS DETALLADAS POR NIVEL DE ACCESO:")
for clasificacion in access_summary['Clasificación de Acceso']:
    municipios_acceso = municipal_coverage_df[municipal_coverage_df['access_classification'] == clasificacion]
    poblacion_promedio = municipios_acceso['total_population_0_19'].mean()
    centros_promedio = municipios_acceso['total_centers'].mean()
    acceso_promedio = municipios_acceso['access_percentage'].mean()

    print(f"   • {clasificacion}: {len(municipios_acceso)} municipios")
    print(f"     - Población promedio 0-19 años: {poblacion_promedio:.0f}")
    print(f"     - Centros educativos promedio: {centros_promedio:.1f}")
    print(f"     - Porcentaje acceso promedio: {acceso_promedio:.1f}%")



📊 DISTRIBUCIÓN DE MUNICIPIOS POR NIVEL DE ACCESO
Distribución por nivel de acceso a ciclos educativos:


,Clasificación de Acceso,Municipios,Porcentaje
0,Acceso Completo,155,86.6
1,Acceso Bueno,24,13.4



📈 ESTADÍSTICAS DETALLADAS POR NIVEL DE ACCESO:
   • Acceso Completo: 155 municipios
     - Población promedio 0-19 años: 8522
     - Centros educativos promedio: 10.7
     - Porcentaje acceso promedio: 100.0%
   • Acceso Bueno: 24 municipios
     - Población promedio 0-19 años: 659
     - Centros educativos promedio: 2.3
     - Porcentaje acceso promedio: 75.0%


In [20]:
# === MUNICIPIOS CON ACCESO LIMITADO O SIN ACCESO ===
print("\n⚠️  MUNICIPIOS CON ACCESO EDUCATIVO LIMITADO")
print("="*50)

municipios_acceso_limitado = municipal_coverage_df[
    municipal_coverage_df['access_classification'].isin(['Acceso Limitado', 'Sin Acceso'])
].copy()

if len(municipios_acceso_limitado) > 0:
    print(f"🔴 MUNICIPIOS CON ACCESO LIMITADO: {len(municipios_acceso_limitado)}")

    # Mostrar top 10 por población
    top_acceso_limitado = municipios_acceso_limitado.nlargest(10, 'total_population_0_19')[
        ['municipality_name', 'total_population_0_19', 'cycles_with_access', 'access_classification', 'total_centers']
    ]

    print("\n   Top 10 por población 0-19 años:")
    for _, municipio in top_acceso_limitado.iterrows():
        ciclos_faltantes = [cycle.replace('_ciclo', '').replace('_', ' ').title() for cycle in municipio.get('missing_cycles', [])]
        print(f"   • {municipio['municipality_name']}")
        print(f"     - Población: {municipio['total_population_0_19']:.0f} hab. 0-19 años")
        print(f"     - Acceso: {municipio['cycles_with_access']}/4 ciclos ({municipio['access_classification']})")
        print(f"     - Centros totales: {municipio['total_centers']}")
        if ciclos_faltantes:
            print(f"     - Ciclos faltantes: {', '.join(ciclos_faltantes)}")
        print()
else:
    print("✅ Todos los municipios tienen acceso a al menos 2 ciclos educativos")



⚠️  MUNICIPIOS CON ACCESO EDUCATIVO LIMITADO
✅ Todos los municipios tienen acceso a al menos 2 ciclos educativos


In [21]:
# === ANÁLISIS DE CICLOS FALTANTES ===
print("\n🔍 ANÁLISIS DE CICLOS EDUCATIVOS FALTANTES")
print("="*50)

# Contar cuántos municipios no tienen cada ciclo
cycles_missing_count = {}
cycle_names = {
    'infantil_i_ciclo': 'Infantil I',
    'infantil_ii_ciclo': 'Infantil II',
    'primaria': 'Primaria',
    'eso': 'ESO'
}

for cycle, name in cycle_names.items():
    municipios_sin_ciclo = (municipal_coverage_df[f'{cycle}_num_centers'] == 0).sum()
    cycles_missing_count[name] = municipios_sin_ciclo

cycles_missing_df = pd.DataFrame({
    'Ciclo Educativo': list(cycles_missing_count.keys()),
    'Municipios sin Oferta': list(cycles_missing_count.values()),
    'Porcentaje': [(count / len(municipal_coverage_df) * 100) for count in cycles_missing_count.values()]
})

print("Municipios que NO tienen oferta por ciclo educativo:")
display(cycles_missing_df.round(1))

# Identificar el ciclo más crítico (mayor número de municipios sin oferta)
ciclo_mas_critico = cycles_missing_df.loc[cycles_missing_df['Municipios sin Oferta'].idxmax()]
print(f"\n🚨 CICLO MÁS CRÍTICO: {ciclo_mas_critico['Ciclo Educativo']}")
print(f"   • {ciclo_mas_critico['Municipios sin Oferta']} municipios sin oferta ({ciclo_mas_critico['Porcentaje']:.1f}%)")



🔍 ANÁLISIS DE CICLOS EDUCATIVOS FALTANTES
Municipios que NO tienen oferta por ciclo educativo:


,Ciclo Educativo,Municipios sin Oferta,Porcentaje
0,Infantil I,24,13.4
1,Infantil II,0,0.0
2,Primaria,0,0.0
3,ESO,0,0.0



🚨 CICLO MÁS CRÍTICO: Infantil I
   • 24 municipios sin oferta (13.4%)


In [22]:
# === COMPARACIÓN: ACCESO VS COBERTURA DE NECESIDADES ===
print("\n🔄 COMPARACIÓN: ACCESO VS COBERTURA DE NECESIDADES")
print("="*60)

# Crear tabla comparativa
comparison_data = []
for _, municipio in municipal_coverage_df.iterrows():
    comparison_data.append({
        'municipality_name': municipio['municipality_name'],
        'access_classification': municipio['access_classification'],
        'access_percentage': municipio['access_percentage'],
        'coverage_classification': 'Excelente' if municipio['coverage_percentage'] >= 100 else
                                  'Buena' if municipio['coverage_percentage'] >= 75 else
                                  'Regular' if municipio['coverage_percentage'] >= 50 else 'Deficiente',
        'coverage_percentage': municipio['coverage_percentage'],
        'total_population_0_19': municipio['total_population_0_19']
    })

comparison_df = pd.DataFrame(comparison_data)

# Matriz de comparación
print("Municipios por combinación de Acceso y Cobertura:")
cross_tab = pd.crosstab(
    comparison_df['access_classification'],
    comparison_df['coverage_classification'],
    margins=True
)
display(cross_tab)

# Casos especiales: Buenos en acceso pero malos en cobertura (y viceversa)
acceso_bueno_cobertura_mala = comparison_df[
    (comparison_df['access_classification'].isin(['Acceso Completo', 'Acceso Bueno'])) &
    (comparison_df['coverage_classification'].isin(['Regular', 'Deficiente']))
]

acceso_malo_cobertura_buena = comparison_df[
    (comparison_df['access_classification'].isin(['Acceso Limitado', 'Acceso Parcial'])) &
    (comparison_df['coverage_classification'].isin(['Excelente', 'Buena']))
]

if len(acceso_bueno_cobertura_mala) > 0:
    print(f"\n🟡 MUNICIPIOS CON BUEN ACCESO PERO COBERTURA INSUFICIENTE: {len(acceso_bueno_cobertura_mala)}")
    print("   (Tienen centros pero capacidad insuficiente para su población)")
    for _, muni in acceso_bueno_cobertura_mala.head(5).iterrows():
        print(f"   • {muni['municipality_name']}: {muni['access_percentage']:.0f}% acceso, {muni['coverage_percentage']:.1f}% cobertura")

if len(acceso_malo_cobertura_buena) > 0:
    print(f"\n🟢 MUNICIPIOS CON POCO ACCESO PERO BUENA COBERTURA: {len(acceso_malo_cobertura_buena)}")
    print("   (Pocos ciclos disponibles pero suficiente capacidad para su población)")
    for _, muni in acceso_malo_cobertura_buena.head(5).iterrows():
        print(f"   • {muni['municipality_name']}: {muni['access_percentage']:.0f}% acceso, {muni['coverage_percentage']:.1f}% cobertura")

print(f"\n💡 INTERPRETACIÓN:")
print(f"   • ACCESO: Mide qué ciclos educativos están disponibles (independiente de capacidad)")
print(f"   • COBERTURA: Mide si la capacidad disponible es suficiente para la población")
print(f"   • Un municipio puede tener buen acceso pero mala cobertura (centros saturados)")
print(f"   • Un municipio puede tener poco acceso pero buena cobertura (pocos ciclos pero suficiente capacidad)")

print(f"\n✅ ANÁLISIS DE ACCESO EDUCATIVO COMPLETADO")



🔄 COMPARACIÓN: ACCESO VS COBERTURA DE NECESIDADES
Municipios por combinación de Acceso y Cobertura:


coverage_classification,Buena,Deficiente,Excelente,Regular,All
access_classification,,,,,
Acceso Bueno,20,3,0,1,24
Acceso Completo,43,55,46,11,155
All,63,58,46,12,179



🟡 MUNICIPIOS CON BUEN ACCESO PERO COBERTURA INSUFICIENTE: 70
   (Tienen centros pero capacidad insuficiente para su población)
   • Daganzo de Arriba: 100% acceso, 25.0% cobertura
   • Colmenar de Oreja: 100% acceso, 25.0% cobertura
   • Moralzarzal: 75% acceso, 0.0% cobertura
   • Guadarrama: 100% acceso, 0.0% cobertura
   • Hoyo de Manzanares: 100% acceso, 25.0% cobertura

💡 INTERPRETACIÓN:
   • ACCESO: Mide qué ciclos educativos están disponibles (independiente de capacidad)
   • COBERTURA: Mide si la capacidad disponible es suficiente para la población
   • Un municipio puede tener buen acceso pero mala cobertura (centros saturados)
   • Un municipio puede tener poco acceso pero buena cobertura (pocos ciclos pero suficiente capacidad)

✅ ANÁLISIS DE ACCESO EDUCATIVO COMPLETADO
